In [1]:
import requests
from bs4 import BeautifulSoup

def getPage(url):
    """
    Utilty function used to get a Beautiful Soup object from a given URL
    """

    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
    try:
        req = session.get(url, headers=headers)
    except requests.exceptions.RequestException:
        return None
    bs = BeautifulSoup(req.text, 'html.parser')
    return bs

### Dealing with different website layouts

In [8]:
import requests

class Content:
    def __init__(self,url, title, body):
        self.url = url
        self.title = title
        self.body = body

def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

import logging

def scrapeNYTimes(url):
    bs = getPage(url)
    try:
        title = bs.find('h1').text
        lines = bs.select('div.StoryBodyCompanionColumn div p')
        body = '\n'.join([line.text for line in lines])
        # return Content(url, title, body)
    except AttributeError as e:
        logging.error("AttributeError: {} for url: {} ".format(e,url))
        print("Title could not be found contact Oscar Karuga")
        return None
    return Content(url, title, body)



def scrapeBrookings(url):
    bs = getPage(url)
    title = bs.find('h1').text
    body = bs.find('div',{'class','post-body'}).text
    return Content(url,title,body)

url = 'https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/'
content = scrapeBrookings(url)
print('Title: {}'.format(content.title))
print('URL: {}'.format(content.url))
print(content.body)

url = 'https://www.nytimes.com/2018/01/25/opinion/sunday/silicon-valley-immortality.html'
content = scrapeNYTimes(url)
print('Title: {}'.format(content.title))
print('URL: {}\n'.format(content.url))
print(content.body)


Title: Delivering inclusive urban access: 3 uncomfortable truths
URL: https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/

The past few decades have been filled with a deep optimism about the role of cities and suburbs across the world. These engines of economic growth host a majority of world population, are major drivers of economic innovation, and have created pathways to opportunities for untold amounts of people.







Jeffrey Gutman

					Former Nonresident Fellow, Global Economy and Development										







Adie Tomer

					Senior Fellow - Brookings Metro 

 Twitter
AdieTomer





But all is not well within our so-called Urban Century. Rapid urbanization, rising gentrification, concentrated poverty, and shortages of basic infrastructure have combined to create spatial inequity in cities and suburbs across the globe. The challenges of housing, moving, and employing so many people have led to longer travel tim

ERROR:root:AttributeError: 'NoneType' object has no attribute 'text' for url: https://www.nytimes.com/2018/01/25/opinion/sunday/silicon-valley-immortality.html 


Title could not be found contact Oscar Karuga


AttributeError: 'NoneType' object has no attribute 'title'

Check how to handle None when scraping data from a website.<br>
trying to handle the error  using try-except block

In [9]:
class Content:
    """
    Common base class for all articles/pages
    """
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        """
        Flexible printing function controls output
        """
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))

class Website:
    """ 
    Contains information about website structure
    """

    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [10]:
import requests
from bs4 import BeautifulSoup


class Crawler:

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        """
        Utilty function used to get a content string from a Beautiful Soup
        object and a selector. Returns an empty string if no object
        is found for the given selector
        """
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''

    def parse(self, site, url):
        """
        Extract content from a given page URL
        """
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()

In [11]:
crawler = Crawler()

siteData = [
    ['O\'Reilly Media', 'http://oreilly.com', 'h1', 'section#product-description'],
    ['Reuters', 'http://reuters.com', 'h1', 'div.StandardArticleBody_body_1gnLA'],
    ['Brookings', 'http://www.brookings.edu', 'h1', 'div.post-body'],
    ['New York Times', 'http://nytimes.com', 'h1', 'div.StoryBodyCompanionColumn div p']
]
websites = []
for row in siteData:
    websites.append(Website(row[0], row[1], row[2], row[3]))

crawler.parse(websites[0], 'http://shop.oreilly.com/product/0636920028154.do')
crawler.parse(
    websites[1], 'http://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0')
crawler.parse(
    websites[2],
    'https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/')
crawler.parse(
    websites[3], 
    'https://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html')

URL: https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
TITLE: Idea to Retire: Old methods of policy education
Idea to Retire: Old methods of policy education
BODY:

Public policy and public affairs schools aim to train competent creators and implementers of government policy. While drawing on the principles that gird our economic and political systems to provide a well-rounded education, like law schools and business schools, policy schools provide professional training. They are quite distinct from graduate programs in political science or economics which aim to train the next generation of academics. As professional training programs, they add value by imparting both the skills which are relevant to current employers, and skills which we know will be relevant as organizations and societies evolve. 
The relevance of the skills that policy programs impart to address problems of today and tomorrow bears further discussion. We are living t

### Crawling through Websites with search

In [12]:
class Content:
    """Common base class for all articles/pages"""

    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.title = title
        self.body = body
        self.url = url

    def print(self):
        """
        Flexible printing function controls output
        """
        print('New article found for topic: {}'.format(self.topic))
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))

In [13]:
class Website:
    """Contains information about website structure"""

    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [14]:
import requests
from bs4 import BeautifulSoup

class Crawler:

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''

    def search(self, topic, site):
        """
        Searches a given website for a given topic and records all pages found
        """
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href']
            # Check to see whether it's a relative or an absolute URL
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url + url)
            if bs is None:
                print('Something was wrong with that page or URL. Skipping!')
                return
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            if title != '' and body != '':
                content = Content(topic, title, body, url)
                content.print()


crawler = Crawler()

siteData = [
    ['O\'Reilly Media', 'http://oreilly.com', 'https://ssearch.oreilly.com/?q=',
        'article.product-result', 'p.title a', True, 'h1', 'section#product-description'],
    ['Reuters', 'http://reuters.com', 'http://www.reuters.com/search/news?blob=', 'div.search-result-content',
        'h3.search-result-title a', False, 'h1', 'div.StandardArticleBody_body_1gnLA'],
    ['Brookings', 'http://www.brookings.edu', 'https://www.brookings.edu/search/?s=',
        'div.list-content article', 'h4.title a', True, 'h1', 'div.post-body']
]
sites = []
for row in siteData:
    sites.append(Website(row[0], row[1], row[2],
                         row[3], row[4], row[5], row[6], row[7]))

topics = ['python', 'data science']
for topic in topics:
    print('GETTING INFO ABOUT: ' + topic)
    for targetSite in sites:
        crawler.search(topic, targetSite)

GETTING INFO ABOUT: python
New article found for topic: python
URL: Housing finance reform: The path forward gets rolling
TITLE: 
The government-sponsored housing finance companies Fannie Mae and Freddie Mac (GSEs) were rescued with large taxpayer injections of capital and placed into legal conservatorship during the financial crisis of 2008. They remain in conservatorship today, controlled on a daily basis by the Federal Housing Finance Agency (FHFA) with the Treasury Department’s legally binding agreement (PSPA) giving Treasury substantial legal authority to go along with its ownership of 79.9 percent of each companies stock. Brookings’ Center on Regulation and Markets convened a diverse group of housing finance experts to discuss the future of the GSEs. This is a recap of that conversation.







Aaron Klein

					Miriam K. Carliner Chair - Economic Studies 

					Senior Fellow - Center on Regulation and Markets 

 Twitter
AaronDKlein





Conservatorship was never meant to be a se

IndexError: list index out of range

### Crawling through Websites with links

In [15]:
class Website:

    def __init__(self, name, url, targetPattern, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.targetPattern = targetPattern
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag


class Content:

    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))

In [17]:
import re


class Crawler:
    def __init__(self, site):
        self.site = site
        self.visited = []

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''

    def parse(self, url):
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, self.site.titleTag)
            body = self.safeGet(bs, self.site.bodyTag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()

    def crawl(self):
        """
        Get pages from website home page
        """
        bs = self.getPage(self.site.url)
        targetPages = bs.findAll('a', href=re.compile(self.site.targetPattern))
        for targetPage in targetPages:
            targetPage = targetPage.attrs['href']
            if targetPage not in self.visited:
                self.visited.append(targetPage)
                if not self.site.absoluteUrl:
                    targetPage = '{}{}'.format(self.site.url, targetPage)
                self.parse(targetPage)


reuters = Website('Reuters', 'https://www.reuters.com', '^(/article/)',
                  False, 'h1', 'div.StandardArticleBody_body_1gnLA')
crawler = Crawler(reuters)
crawler.crawl()

### Crawling multiple page types

In [18]:
class Website:
    """Common base class for all articles/pages"""

    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [19]:
class Product(Website):
    """Contains information for scraping a product page"""

    def __init__(self, name, url, titleTag, productNumber, price):
        Website.__init__(self, name, url, TitleTag)
        self.productNumberTag = productNumberTag
        self.priceTag = priceTag

class Article(Website):
    """Contains information for scraping an article page"""

    def __init__(self, name, url, titleTag, bodyTag, dateTag):
        Website.__init__(self, name, url, titleTag)
        self.bodyTag = bodyTag
        self.dateTag = dateTag

In [23]:
#def parsePage(url):
    
    #if '/ideas/' in url:
        

#oreilly = Website('O\'Reilly', 'https://oreilly.com', 'h1' '')   

## Web Scraping with Python using scrapy framework

scrapy startproject wikiSpider <br><br>
After creating files in the wikiSpider/wikiSpider/spider folder eg article.py

run using :<br>
"""
scrapy runspider article.py
"""

In [24]:
# scrapy code 
import scrapy

class ArticleSpider(scrapy.Spider):
    name='article'

    def start_requests(self): 
        urls=[
                'http://en.wikipedia.org/wiki/Python_'
                '%28programming_language%29',
                'https://en.wikipedia.org/wiki/Functional_programming',
                'https://en.wikipedia.org/wiki/Monty_Python']

        return [scrapy.Request(url=url, callback=self.parse) 
            for url in urls]

    def parse(self, response):
        url = response.url
        title = response.css('title::text').extract_first() 
        # body::text
        #body = response.xpath('body::text').extract()
        print('URL is: {}'.format(url))
        print('Title is: {}'.format(title))
        #print('Body is: {}'.format(body))

crawling pages using scrapy and saving the data in either csv, json or xml format:<br><br>
    scrapy runspider articleItems.py -o articles.csv -t csv<br><br>
    scrapy runspider articleItems.py -o articles.json -t json<br><br>
    scrapy runspider articleItems.py -o articles.xml -t xml<br><br>


In [28]:
import pandas as pd 

# data scraped using article_item.py
data  = pd.read_csv('/Users/oscar/desktop/Android-kotlin/DataScience/web_scrapping/wikiSpider/wikiSpider/spiders/articles.csv')
data.head(10)

,lastUpdated,text,title,url
0,"This page was last edited on 15 June 2022, at...","Software developer,\n,Ken Manheimer, has been ...",Ken Manheimer - Wikipedia,https://en.wikipedia.org/wiki/Ken_Manheimer
1,"This page was last edited on 26 June 2022, at...",""",Homesteading the Noosphere,"" (abbreviated ,H...",Homesteading the Noosphere - Wikipedia,https://en.wikipedia.org/wiki/Homesteading_the...
2,This page was last edited on 16 September 202...,Freely accessible creation and refinement of c...,Open-source software development - Wikipedia,https://en.wikipedia.org/wiki/Open-source_soft...
3,"This page was last edited on 10 January 2023,...",Title given to a small number of open-source s...,Benevolent dictator for life - Wikipedia,https://en.wikipedia.org/wiki/Benevolent_dicta...
4,"This page was last edited on 11 January 2023,...","Political system,\n,.mw-parser-output .hatnote...",Benevolent dictatorship - Wikipedia,https://en.wikipedia.org/wiki/Benevolent_dicta...
5,"This page was last edited on 5 December 2022,...","New program, and line of software development,...",Fork (software development) - Wikipedia,https://en.wikipedia.org/wiki/Fork_(software_d...
6,"This page was last edited on 3 January 2023, ...","Subculture of individuals,\n,.mw-parser-output...",Hacker culture - Wikipedia,https://en.wikipedia.org/wiki/Hacker_culture
7,This page was last edited on 17 November 2022...,"Dutch programmer and creator of Python,\n,.mw-...",Guido van Rossum - Wikipedia,https://en.wikipedia.org/wiki/Guido_van_Rossum
8,This page was last edited on 21 September 202...,"Free software,\n,.mw-parser-output .infobox-su...",Spacemacs - Wikipedia,https://en.wikipedia.org/wiki/Spacemacs
9,"This page was last edited on 10 January 2023,...","Russian-Canadian programmer (born 1994),\n,.mw...",Vitalik Buterin - Wikipedia,https://en.wikipedia.org/wiki/Vitalik_Buterin


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   lastUpdated  124 non-null    object
 1   text         124 non-null    object
 2   title        124 non-null    object
 3   url          124 non-null    object
dtypes: object(4)
memory usage: 4.0+ KB


In [30]:
data.shape

(124, 4)